In [ ]:
# Импорт необходимых библиотек
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Загрузка данных
data = pd.read_csv('flight_delay.csv')

# Предварительный анализ данных
print(data.head())
print(data.info())
print(data.describe())


departureAirportColumn = data.columns[0]
destinationAirportColumn = data.columns[2]
scheduledDepartureTimeColumn = data.columns[1]
scheduledArrivalTimeColumn = data.columns[3]
delayColumn=data.columns[4]

In [3]:
# Обработка строковых данных
le = LabelEncoder()
data[departureAirportColumn] = le.fit_transform(data[departureAirportColumn])
data[destinationAirportColumn] = le.fit_transform(data[destinationAirportColumn])

# Преобразование временных меток в datetime
data[scheduledDepartureTimeColumn] = pd.to_datetime(data[scheduledDepartureTimeColumn])
data[scheduledArrivalTimeColumn] = pd.to_datetime(data[scheduledArrivalTimeColumn])

# Извлечение признаков из временных меток
data['Year'] = data[scheduledDepartureTimeColumn].dt.year
data['Month'] = data[scheduledDepartureTimeColumn].dt.month
data['Day'] = data[scheduledDepartureTimeColumn].dt.day
data['Day of week'] = data[scheduledDepartureTimeColumn].dt.dayofweek

print(data.columns)

Index(['Depature Airport', 'Scheduled depature time', 'Destination Airport',
       'Scheduled arrival time', 'Delay', 'Year', 'Month', 'Day',
       'Day of week'],
      dtype='object')


In [4]:
# Рассчет продолжительности полета
data['Flight duration'] = (data[scheduledArrivalTimeColumn] - data[scheduledDepartureTimeColumn]).dt.total_seconds() / 60

# Разделение данных на обучающую и тестовую выборки
train_data = data[data['Year'] < 2018]
test_data = data[data['Year'] == 2018]

X_train = train_data[[departureAirportColumn, destinationAirportColumn, 'Month', 'Day', 'Day of week', 'Flight duration']]
y_train = train_data['Delay']
X_test = test_data[[departureAirportColumn, destinationAirportColumn, 'Month', 'Day', 'Day of week', 'Flight duration']]
y_test = test_data['Delay']

# Обнаружение и удаление выбросов (используем межквартильный размах на одном месяце)
month_data = data[data['Month'] == 1]  # пример для января
q1 = month_data['Delay'].quantile(0.25)
q3 = month_data['Delay'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr
data = data[(data['Delay'] >= lower_bound) & (data['Delay'] <= upper_bound)]

# Обучение моделей машинного обучения
models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(),
    'Random Forest': RandomForestRegressor()
}

In [ ]:
# Оценка моделей и сравнение их результатов
results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    train_mse = mean_squared_error(y_train, y_pred_train)
    test_mse = mean_squared_error(y_test, y_pred_test)
    train_r2 = r2_score(y_train, y_pred_train)
    test_r2 = r2_score(y_test, y_pred_test)
    results[name] = {
        'Train MSE': train_mse,
        'Test MSE': test_mse,
        'Train R2': train_r2,
        'Test R2': test_r2
    }
    print(f"{name} - Train MSE: {train_mse:.2f}, Test MSE: {test_mse:.2f}, Train R2: {train_r2:.2f}, Test R2: {test_r2:.2f}")

# Сравнение моделей
results_df = pd.DataFrame(results).T
print(results_df)

Linear Regression - Train MSE: 2148.70, Test MSE: 1615.50, Train R2: 0.00, Test R2: -0.01
Ridge Regression - Train MSE: 2148.70, Test MSE: 1615.50, Train R2: 0.00, Test R2: -0.01


In [ ]:
# Визуализация результатов
plt.figure(figsize=(10, 5))
sns.barplot(data=results_df.reset_index(), x='index', y='Test R2', palette='viridis')
plt.title('Comparison of Model Test R2 Scores')
plt.xlabel('Model')
plt.ylabel('Test R2 Score')
plt.show()